In [1]:
import copy

import compound_freecad as FC_Cmp
import core_freecad_ops as FC_Ops
import core_freecad_commands as FC_Cmd

import compound_cameo as MD_Cmp
import core_cameo_ops as MD_Ops
import core_cameo_commands as MD_Cmd

import recipe_oop as RC_OOP
import recipe_systems as RC_Sys

import uuid

# Section 0 - Load up Libraries and Common Elements

Find elements already in the base model and provide handles to them.

In [2]:
library_element_ids = \
    {
        'Integer ValueType': '\"_16_5_1_12c903cb_1245415335546_8641_4088\"',
        'Boolean ValueType': '\"_16_5_1_12c903cb_1245415335546_39033_4086\"'
    }
library_element_vars = \
    {
        'Integer ValueType': 'intVal',
        'Boolean ValueType': 'boolVal'
    }

lib_strings = []

for key, val in library_element_ids.items():
    lib_strings.append(
        library_element_vars[key] + ' = \n' +
            '    com.nomagic.magicdraw.core.Application.getInstance().getProject().getElementByID(' +
        val + ');\n'
    )


# Section 1 - Logical Architecture for Model Editing

Capture functions related to editing models (adding literal values to elements, references, and so on).

## Section 1.1 - Business Objects

Describe the things that the functions are meant to work on. A lot of the functional descriptions will involve changing the states of these objects.

### Section 1.1.1 - Model Element and Connections

The base object is of course the model object itself. The model object is this approach is basically a container for a list of literal values and can reference other such containers to support building a composite or aggregate set of values on which to compute.

In [3]:
model_obj = MD_Cmp.T_Sysml_Block(0)
model_obj.attributes_dict.update({'name': 'Model Object'})

key_val_obj = MD_Cmp.T_Sysml_Block(0)
key_val_obj.attributes_dict.update({'name': 'Literal Value'})

pointer_obj = MD_Cmp.T_Sysml_Block(0)
pointer_obj.attributes_dict.update({'name': 'Model Object Pointer'})

model_obj.part_types_dict.update(
    {
        'Values' : key_val_obj,
        'References' : pointer_obj
    }
)

biz_objs = [model_obj, key_val_obj, pointer_obj]

## Section 1.2 - Functions

The simple functions of editing and dealing with two-way references are captured here.

In [4]:
create_function = RC_Sys.T_Component_Function(0)
create_function.name = 'Create Modeling Object'
create_function.owner_var = 'pkg_id_1'
create_function.state_outcome_name = 'Created'
create_function.state_target = model_obj.generate_cameo_handle()

outer_ref_function = RC_Sys.T_Component_Function(0)
outer_ref_function.name = 'Create Syntactically Correct Outgoing Reference'
outer_ref_function.owner_var = 'pkg_id_1'
outer_ref_function.state_outcome_name = 'Reference Added'
outer_ref_function.state_target = model_obj.generate_cameo_handle()

outer_ref_rm_function = RC_Sys.T_Component_Function(0)
outer_ref_rm_function.name = 'Remove Outgoing Reference in Syntactically Correct Way'
outer_ref_rm_function.owner_var = 'pkg_id_1'
outer_ref_rm_function.state_outcome_name = 'Reference Removed'
outer_ref_rm_function.state_target = model_obj.generate_cameo_handle()

# Section 2 - Logical Architecture for Model Storage and Collaboration

This section aims to capture the functionality around storing model data and making it available for collaboration.

In [5]:
runtime_obj = MD_Cmp.T_Sysml_Block(0)
runtime_obj.attributes_dict.update({'name': 'Runtime Model Class'})

In [6]:
schema_obj = MD_Cmp.T_Sysml_Block(0)
schema_obj.attributes_dict.update({'name': 'Database Instructions'})

editor_obj = MD_Cmp.T_Sysml_Block(0)
editor_obj.attributes_dict.update({'name': 'Runtime Editor Provider for Model'})

editor_cmp_obj = MD_Cmp.T_Sysml_Block(0)
editor_cmp_obj.attributes_dict.update({'name': 'Components for Model Editor'})

In [7]:
function_outputs = []
function_outputs.extend(runtime_obj.generate_for_cameo_abstract('pkg_id_1'))
function_outputs.extend(schema_obj.generate_for_cameo_abstract('pkg_id_1'))
function_outputs.extend(editor_obj.generate_for_cameo_abstract('pkg_id_1'))
function_outputs.extend(editor_cmp_obj.generate_for_cameo_abstract('pkg_id_1'))

In [8]:
functions = []

ind_author_function = RC_Sys.T_Component_Function(0)
ind_author_function.name = 'Individually Author Modeling Object'
ind_author_function.state_outcome_name = 'Individual Edits Taken'
ind_author_function.state_target = model_obj.generate_cameo_handle()

collab_author_function = RC_Sys.T_Component_Function(0)
collab_author_function.name = 'Collaboratively Author Modeling Object'
collab_author_function.state_outcome_name = 'Collaborative Edits Taken'
collab_author_function.state_target = model_obj.generate_cameo_handle()

store_function = RC_Sys.T_Component_Function(0)
store_function.name = 'Preserve Modeling Object'
store_function.state_outcome_name = 'Preserved'
store_function.state_target = model_obj.generate_cameo_handle()

analyze_function = RC_Sys.T_Component_Function(0)
analyze_function.name = 'Analyze Modeling Object'
analyze_function.state_outcome_name = 'Analyzed'
analyze_function.state_target = model_obj.generate_cameo_handle()

runtime_function = RC_Sys.T_Component_Function(0)
runtime_function.name = 'Generate Class Definitions for Stateful Work'
runtime_function.flow_outcome_name = 'Model-based Classes'
runtime_function.flow_outcome_type = runtime_obj.generate_cameo_handle()

schema_function = RC_Sys.T_Component_Function(0)
schema_function.name = 'Generate Database Schemas for Storage'
schema_function.flow_outcome_name = 'Model-based Schemas'
schema_function.flow_outcome_type = schema_obj.generate_cameo_handle()

editor_function = RC_Sys.T_Component_Function(0)
editor_function.name = 'Generate Editor Helper Classes'
editor_function.flow_outcome_name = 'Model-based Editor Helper Classes'
editor_function.flow_outcome_type = editor_obj.generate_cameo_handle()

editor_function = RC_Sys.T_Component_Function(0)
editor_function.name = 'Generate Editor Component Classes'
editor_function.flow_outcome_name = 'Editor Components'
editor_function.flow_outcome_type = editor_cmp_obj.generate_cameo_handle()

functions.append(create_function)
functions.append(outer_ref_function)
functions.append(outer_ref_rm_function)

functions.append(ind_author_function)
functions.append(collab_author_function)
functions.append(store_function)
functions.append(analyze_function)
functions.append(runtime_function)
functions.append(schema_function)
functions.append(editor_function)

In [9]:
integer_obj = MD_Cmp.T_Sysml_Block(0)
integer_obj.attributes_dict.update({'name': 'Integer'})

collection_obj = MD_Cmp.T_Sysml_Block(0)
collection_obj.attributes_dict.update({'name': 'java.util.Collection'})

obj_obj = MD_Cmp.T_Sysml_Block(0)
obj_obj.attributes_dict.update({'name': 'java.lang.Object'})

behavior_test = []

behavior_test.extend(integer_obj.generate_for_cameo_abstract('pkg_id_1'))
behavior_test.extend(collection_obj.generate_for_cameo_abstract('pkg_id_1'))
behavior_test.extend(obj_obj.generate_for_cameo_abstract('pkg_id_1'))

# Section 3 - Select Technology Models

Models that are used to understand the core of key technologies that support the larger system.

TODO: Create a version of this that has swimlanes for different classes of inheritance to show how each contributes the methods used by the ultimate user-facing model class in the EMF.

## Section 3.1 Eclipse Modeling Framework List Handling

This section creates model elements for capturing the EMF and its list facilities for references.

## Section 3.1.1 Key Classes and Hierarchy

In [10]:
notifying_list_int_obj = MD_Cmp.T_Sysml_Interface_Block(0)
notifying_list_int_obj.attributes_dict.update({'name': 'NotifyingList'})

basic_list_obj = MD_Cmp.T_Sysml_Block(0)
basic_list_obj.attributes_dict.update({'name': 'BasicEList'})

abstract_elist_obj = RC_OOP.T_Class(0)
abstract_elist_obj.attributes_dict.update({'name': 'AbstractEList'})

notifying_list_obj = RC_OOP.T_Class(0)
notifying_list_obj.attributes_dict.update({'name': 'NotifyingListImpl'})

eobj_with_inverse_resolving_elist_obj = MD_Cmp.T_Sysml_Block(0)
eobj_with_inverse_resolving_elist_obj.attributes_dict.update({'name': 'EObjectWithInverseResolvingEList'})

eobj_with_inverse_elist_obj = MD_Cmp.T_Sysml_Block(0)
eobj_with_inverse_elist_obj.attributes_dict.update({'name': 'EObjectWithInverseEList'})

eobj_elist_obj = MD_Cmp.T_Sysml_Block(0)
eobj_elist_obj.attributes_dict.update({'name': 'EObjectEList'})

ecore_elist_obj = MD_Cmp.T_Sysml_Block(0)
ecore_elist_obj.attributes_dict.update({'name': 'EcoreEList'})

notifying_list_obj.general_types.append(notifying_list_int_obj)
notifying_list_obj.general_types.append(basic_list_obj)

basic_list_obj.general_types.append(abstract_elist_obj)

eobj_with_inverse_resolving_elist_obj.general_types.append(eobj_with_inverse_elist_obj)

eobj_with_inverse_elist_obj.general_types.append(eobj_elist_obj)

eobj_elist_obj.general_types.append(ecore_elist_obj)

ecore_elist_obj.general_types.append(notifying_list_obj)

tech_objs = \
[
    abstract_elist_obj,
    notifying_list_int_obj,
    basic_list_obj,
    notifying_list_obj,
    ecore_elist_obj,
    eobj_elist_obj,
    eobj_with_inverse_elist_obj,
    eobj_with_inverse_resolving_elist_obj
]

Also some key Ecore objects to handle.

In [11]:
eobj = MD_Cmp.T_Sysml_Block(0)
eobj.attributes_dict.update({'name': 'EObject'})

estrf = MD_Cmp.T_Sysml_Block(0)
estrf.attributes_dict.update({'name': 'EStructuralFeature'})

ecore_base_objs = []
ecore_base_objs.append(eobj)
ecore_base_objs.append(estrf)

## Section 3.1.2 - Key Methods for Classes

List out the methods as attached to the classes and also behaviors inside them.

First this goes through addAllUnique and remove as core low-level functions.

TODO: Reorganize in order to get proper ordering of references so that operations, methods, and call behavior methods are built in the right order. May need to have the ability to build a signature for Code Blocks and then fill in details.

Start with the code blocks that will have internal details added to show traceability. This traces out the call track for an add command:
addAll (AbstractEList)
addAllUnique variant 4 (AbstractEList, overridden by NotifyingListImpl)
addAllUnique variant 2 (NotifyingListImpl)
inverseAdd (EcoreEList)

In [12]:
wrapped_blocks = []

add_all = RC_OOP.T_Code_Block()
add_all.name = 'AddAll for AbstractEList'

collection_starter = RC_OOP.T_Object_State()
collection_starter.working_name = 'collection'
collection_starter.object = collection_obj

add_all_collection_in = RC_OOP.T_Code_Input()
add_all_collection_in.name = 'collection'
add_all_collection_in.para_type = collection_obj

add_all.passed_parameters.append(add_all_collection_in)

#add_all.passed_parameter_dict.update({'collection': collection_obj})
add_all.passed_parameter_state_dict.update({'collection': collection_starter})

add_all_unique_4_abstr_m = RC_OOP.T_Code_Block()
add_all_unique_4_abstr_m.name = 'AddAllUnique Variant 4 for AbstractEList'

collection_starter_aau_col4 = RC_OOP.T_Object_State()
collection_starter_aau_col4.working_name = 'collection'
collection_starter_aau_col4.object = collection_obj

add_all_unique4_collection_in = RC_OOP.T_Code_Input()
add_all_unique4_collection_in.name = 'collection'
add_all_unique4_collection_in.para_type = collection_obj

add_all_unique_4_abstr_m.passed_parameters.append(add_all_unique4_collection_in)

#add_all_unique_4_abstr_m.passed_parameter_dict.update(
#    {
#        'collection': collection_obj
#    }
#)
add_all_unique_4_abstr_m.passed_parameter_state_dict.update(
    {
        'collection': collection_starter_aau_col4
    }
)

add_all_unique_2_not_m = RC_OOP.T_Code_Block()
add_all_unique_2_not_m.name = 'AddAllUnique Variant 2 for NotifyingListImpl'

#abstract_elist_obj.\
#        operation_methods_dict.update({'addAll': 
#                                       add_all.generate_cameo_handle()})

collection_starter_aau_ind2 = RC_OOP.T_Object_State()
collection_starter_aau_ind2.working_name = 'index'
collection_starter_aau_ind2.object = collection_obj
collection_starter_aau_col2 = RC_OOP.T_Object_State()
collection_starter_aau_col2.working_name = 'collection'
collection_starter_aau_col2.object = collection_obj

add_all_unique2_collection_in = RC_OOP.T_Code_Input()
add_all_unique2_collection_in.name = 'index'
add_all_unique2_collection_in.para_type = 'intVal'

add_all_unique2_collection_col = RC_OOP.T_Code_Input()
add_all_unique2_collection_col.name = 'collection'
add_all_unique2_collection_col.para_type = collection_obj

add_all_unique_2_not_m.passed_parameters.append(add_all_unique2_collection_in)
add_all_unique_2_not_m.passed_parameters.append(add_all_unique2_collection_col)

#add_all_unique_2_not_m.passed_parameter_dict.update(
#    {
#        'index': 'intVal',
#        'collection': collection_obj
#    }
#)
add_all_unique_2_not_m.passed_parameter_state_dict.update(
    {
        'index': collection_starter_aau_ind2,
        'collection': collection_starter_aau_col2
    }
)

wrapped_blocks.append(add_all)
wrapped_blocks.append(add_all_unique_4_abstr_m)
wrapped_blocks.append(add_all_unique_2_not_m)

In [13]:
method_objs = []

common_list_sigs = ['AbstractEList', 'BasicEList', 'NotifyingList', 'NotifyingListImpl']

for cl in common_list_sigs:

    add_all_unique1 = MD_Cmp.T_Sysml_Opaque_Behavior()
    add_all_unique1.name = 'AddAllUnique1 for ' + cl

    add_all_unique1.parameters_dict.update({'index': 'index'})
    add_all_unique1.parameters_dict.update({'objects': 'objects'})
    add_all_unique1.parameters_dict.update({'start': 'start'})
    add_all_unique1.parameters_dict.update({'end': 'end'})
    add_all_unique1.parameters_dict.update({'result': 'result'})

    add_all_unique1.parameter_types_dict.update({'index': library_element_vars['Integer ValueType']})
    add_all_unique1.parameter_types_dict.update({'objects': obj_obj.generate_cameo_handle()})
    add_all_unique1.parameter_types_dict.update({'start': library_element_vars['Integer ValueType']})
    add_all_unique1.parameter_types_dict.update({'end': library_element_vars['Integer ValueType']})
    add_all_unique1.parameter_types_dict.update({'result': library_element_vars['Boolean ValueType']})

    add_all_unique1.parameter_directions_dict.update({'index': 'in'})
    add_all_unique1.parameter_directions_dict.update({'objects': 'in'})
    add_all_unique1.parameter_directions_dict.update({'start': 'in'})
    add_all_unique1.parameter_directions_dict.update({'end': 'in'})
    add_all_unique1.parameter_directions_dict.update({'result': 'return'})
    
    add_all_unique1.paramter_lower_mults_dict.update({'objects': 0})
    add_all_unique1.paramter_upper_mults_dict.update({'objects': -1})

    add_all_unique2 = MD_Cmp.T_Sysml_Opaque_Behavior()
    add_all_unique2.name = 'AddAllUnique2 for ' + cl

    add_all_unique2.parameters_dict.update({'index': 'index'})
    add_all_unique2.parameters_dict.update({'collection': 'collection'})
    add_all_unique2.parameters_dict.update({'result': 'result'})

    add_all_unique2.parameter_types_dict.update({'index': library_element_vars['Integer ValueType']})
    add_all_unique2.parameter_types_dict.update({'collection': collection_obj.generate_cameo_handle()})
    add_all_unique2.parameter_types_dict.update({'result': library_element_vars['Boolean ValueType']})

    add_all_unique2.parameter_directions_dict.update({'index': 'in'})
    add_all_unique2.parameter_directions_dict.update({'collection': 'in'})
    add_all_unique2.parameter_directions_dict.update({'result': 'return'})

    add_all_unique3 = MD_Cmp.T_Sysml_Opaque_Behavior()
    add_all_unique3.name = 'AddAllUnique3 for ' + cl

    add_all_unique3.parameters_dict.update({'objects': 'objects'})
    add_all_unique3.parameters_dict.update({'start': 'start'})
    add_all_unique3.parameters_dict.update({'end': 'end'})
    add_all_unique3.parameters_dict.update({'result': 'result'})
    
    add_all_unique3.paramter_lower_mults_dict.update({'objects': 0})
    add_all_unique3.paramter_upper_mults_dict.update({'objects': -1})

    add_all_unique3.parameter_types_dict.update({'objects': obj_obj.generate_cameo_handle()})
    add_all_unique3.parameter_types_dict.update({'start': library_element_vars['Integer ValueType']})
    add_all_unique3.parameter_types_dict.update({'end': library_element_vars['Integer ValueType']})
    add_all_unique3.parameter_types_dict.update({'result': library_element_vars['Boolean ValueType']})

    add_all_unique3.parameter_directions_dict.update({'objects': 'in'})
    add_all_unique3.parameter_directions_dict.update({'start': 'in'})
    add_all_unique3.parameter_directions_dict.update({'end': 'in'})
    add_all_unique3.parameter_directions_dict.update({'result': 'return'})

    add_all_unique4 = MD_Cmp.T_Sysml_Opaque_Behavior()
    add_all_unique4.name = 'AddAllUnique4 for ' + cl

    add_all_unique4.parameters_dict.update({'collection': 'collection'})
    add_all_unique4.parameters_dict.update({'result': 'result'})

    add_all_unique4.parameter_types_dict.update({'collection': collection_obj.generate_cameo_handle()})
    add_all_unique4.parameter_types_dict.update({'result': library_element_vars['Boolean ValueType']})

    add_all_unique4.parameter_directions_dict.update({'collection': 'in'})
    add_all_unique4.parameter_directions_dict.update({'result': 'return'})

    remove = MD_Cmp.T_Sysml_Opaque_Behavior()
    remove.name = 'Remove for ' + cl

    remove.parameters_dict.update({'index': 'index'})
    remove.parameters_dict.update({'result': 'result'})

    remove.parameter_types_dict.update({'index': library_element_vars['Integer ValueType']})
    remove.parameter_types_dict.update({'result': obj_obj.generate_cameo_handle()})

    remove.parameter_directions_dict.update({'index': 'in'})
    remove.parameter_directions_dict.update({'result': 'return'})
    
    obj_to_update = None
    
    ['AbstractEList', 'BasicEList', 'NotifyingList', 'NotifyingListImpl']
    
    if cl == 'AbstractEList':
        obj_to_update = abstract_elist_obj
    elif cl == 'BasicEList':
        obj_to_update = basic_list_obj
    elif cl == 'NotifyingList':
        obj_to_update = notifying_list_int_obj
    elif cl == 'NotifyingListImpl':
        obj_to_update = notifying_list_obj
    
    obj_to_update.\
        operation_methods_dict.update({'addAllUnique Variant 1': 
                                       add_all_unique1.generate_cameo_handle()})
    if cl != 'NotifyingListImpl':
        obj_to_update.\
            operation_methods_dict.update({'addAllUnique Variant 2': 
                                           add_all_unique2.generate_cameo_handle()})
    else:
        pass
        #obj_to_update.\
        #    operation_methods_dict.update({'addAllUnique Variant 2': 
        #                                   add_all_unique_2_not_m.generate_cameo_handle()})
        
    obj_to_update.\
        operation_methods_dict.update({'addAllUnique Variant 3': 
                                       add_all_unique3.generate_cameo_handle()})
    if cl != 'AbstractEList':
        obj_to_update.\
            operation_methods_dict.update({'addAllUnique Variant 4': 
                                           add_all_unique4.generate_cameo_handle()})
    else:
        pass
        #obj_to_update.\
        #    operation_methods_dict.update({'addAllUnique Variant 4': 
        #                                   add_all_unique_4_abstr_m.generate_cameo_handle()})
        
    obj_to_update.\
        operation_methods_dict.update({'remove': 
                                       remove.generate_cameo_handle()})
    
    method_objs.append(add_all_unique1)
    method_objs.append(add_all_unique2)
    method_objs.append(add_all_unique3)
    method_objs.append(add_all_unique4)
    method_objs.append(remove)

Need to pre-bake the Blocks for now to get access to operations for the methods here. Need to left this to the OOP Recipe level rather than the SysML level.

In [14]:
add_all_unique4_sig = RC_OOP.T_Method_Signature()
add_all_unique4_sig.name = 'addAllUnique Variant 4'
add_all_unique4_sig.owning_class = abstract_elist_obj
add_all_unique4_sig.implementation = add_all_unique_4_abstr_m

add_all_unique2_sig = RC_OOP.T_Method_Signature()
add_all_unique2_sig.name = 'addAllUnique Variant 2'
add_all_unique2_sig.owning_class = notifying_list_obj
add_all_unique2_sig.implementation = add_all_unique_2_not_m

add_all_sig = RC_OOP.T_Method_Signature()
add_all_sig.name = 'addAll'
add_all_sig.owning_class = notifying_list_obj
add_all_sig.implementation = add_all

abstract_elist_obj.methods.append(add_all_unique4_sig)
notifying_list_obj.methods.append(add_all_unique2_sig)
abstract_elist_obj.methods.append(add_all_sig)

genned_tech_objs = []

for tech in tech_objs:
    for dikt in tech.generate_for_cameo_abstract('pkg_id_4'):
        genned_tech_objs.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
add_all_unique4_call = RC_OOP.T_Object_Method_Call()
add_all_unique4_call.method_to_call = add_all_unique4_sig
add_all_unique4_call.object_states_used.append(
        collection_starter
)
add_all_unique4_call.object_parameters_input.append(
    add_all_unique4_collection_in
)

add_all_unique2_call = RC_OOP.T_Object_Method_Call()
add_all_unique2_call.method_to_call = add_all_unique2_sig
add_all_unique2_call.object_states_used.append(
        collection_starter_aau_col4
)

add_all_unique2_call.object_parameters_input.append(
    add_all_unique2_collection_col
)

add_all.method_calls.append(add_all_unique4_call)

add_all_unique_4_abstr_m.method_calls.append(add_all_unique2_call)

Apply additional core methods (such as general addAll).

Reflective capabilities for the lists are brought in to EcoreEList.

In [15]:
reflective_methods = []

estruct = MD_Cmp.T_Sysml_Opaque_Behavior()
estruct.name = 'GetEStructuralFeature'

estruct.parameters_dict.update({'result': 'result'})

estruct.parameter_types_dict.update({'result': estrf.generate_cameo_handle()})

estruct.parameter_directions_dict.update({'result': 'return'})

get_eobj = MD_Cmp.T_Sysml_Opaque_Behavior()
get_eobj.name = 'GetEObject'

get_eobj.parameters_dict.update({'result': 'result'})

get_eobj.parameter_types_dict.update({'result': eobj.generate_cameo_handle()})

get_eobj.parameter_directions_dict.update({'result': 'return'})

ecore_elist_obj.\
        operation_methods_dict.update({'getEObject': 
                                       get_eobj.generate_cameo_handle()})
ecore_elist_obj.\
        operation_methods_dict.update({'getEStructuralFeature': 
                                       estruct.generate_cameo_handle()})

reflective_methods.append(get_eobj)
reflective_methods.append(estruct)

Set up redefinitions to get a proper trace when running through examples.

# Section 4 - Unit Test / "Tech Rehearsal"

This section lays out test-specific contexts, objects, and behavioral usages to act as unit tests in some ways, but more importantly as "Tech Rehearsals" that allow for the simulation, playback, and study of various use cases of the key technology components to build up understanding.

## Section 4.1 - Responding to Commands

Go through real and hypothetical edit commands and how the lower level frameworks respond to them in that layer.

### Section 4.1.1 - Add Command Demonstrations

Here we can cycle through a number of cases and scenarios of different kinds of EMF reference attributes and their attendant lists when items are to be added to them in different ways.

# Usage Flow

In [16]:
class T_User_Process():
    
    def __init__():
        self.user_steps = []
        self.process_goal = None

Develop a set of input parameters with objects and aligning initial states.

In [17]:
build_list = []
build_list = MD_Cmd.access_libraries()
build_list.extend(MD_Cmd.cameo_build_session_start())
build_list.extend(lib_strings)
stringified = []

for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Base Package",
    order_start=1,
    owner_obj='modelBase'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Model Functions",
    order_start=2,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))        

for dikt in MD_Cmp.cameo_package_create_dicts(
    name="EMF Metaobjects",
    order_start=3,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in MD_Cmp.cameo_package_create_dicts(
    name="EMF List Processing",
    order_start=4,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Technical Rehearsal",
    order_start=5,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

stringified.append("// Add Business Objects below:")

for biz in biz_objs:
    for dikt in biz.generate_for_cameo_abstract('pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

stringified.append("// Add Functional Outputs below:")

for dikt in function_outputs:
    stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

stringified.append("// Add Functions below:")
    
for fun in functions:
    for dikt in fun.generate_for_cameo_abstract('pkg_id_2'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
stringified.append("// Add Function Test below:")
        
for dikt in behavior_test:
    stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

stringified.append("// Add Ecore Base Objects below:")
for ebo in ecore_base_objs:
    for dikt in ebo.generate_for_cameo_abstract('pkg_id_3'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
    
stringified.append("// Add Reflective Methods below:")
for ref in reflective_methods:
    for dikt in ref.generate_for_cameo_abstract('pkg_id_4'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
    
stringified.append("// Add Method Objects below:")
for meth in method_objs:
    for dikt in meth.generate_for_cameo_abstract('pkg_id_4'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

stringified.append("// Add Wrapped Blocks (signatures only) below:")
for wb in wrapped_blocks:
    for dikt in wb.generate_for_cameo_sig_only('pkg_id_4'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
stringified.append("// Add Tech Objects below:")        
stringified.extend(genned_tech_objs)

stringified.append("// Add Wrapped Blocks (internals) below:")
for wb in wrapped_blocks:
    for dikt in wb.generate_for_cameo_internals_only('pkg_id_4'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in ind_author_function.generate_for_cameo_abstract('pkg_id_2'):
    stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
    

cameo references
{'collection': 'pin_id_af69fb37_b7f5_446d_8632_1ca5a41e80f6'}
cameo references
{'index': 'pin_id_e14ad9a7_8afa_4243_abf0_f1b2b5227103', 'collection': 'pin_id_1c076efa_6ad2_4140_8bcd_a1fd52e72f69'}


In [18]:
build_list.extend(stringified)

build_list.extend(MD_Cmd.cameo_build_session_end())

In [19]:
create_function.cameo_references

{'Function': 'function_id_c65f2681_cf48_492e_8872_03ee6a9bb18e',
 'State Signal': 'signal_id_1960ffd6_e56c_4017_a29a_3c39a6125a36',
 'State Target Parameter': 'parameter_id_3639532e_41ce_44d6_bc19_3e7aaea05c91',
 'State Target Node': 'apn_id_3639532e_41ce_44d6_bc19_3e7aaea05c91',
 'State Send Signal Action': 'ssa_id_1960ffd6_e56c_4017_a29a_3c39a6125a36',
 'State SSA Pin': 'ssa_input_id_1960ffd6_e56c_4017_a29a_3c39a6125a36',
 'Initial Node': 'initial_id_c65f2681_cf48_492e_8872_03ee6a9bb18e',
 'Initial Control': 'initial_control_id_c65f2681_cf48_492e_8872_03ee6a9bb18e',
 'Flow Parameter': 'flow_parameter_id_a65354e3_397f_4599_b0af_95475d4d0c56'}

In [20]:
f = open("cameo_ops_instructions.txt", "w")
for build in build_list:
    f.write(build + '\n')
f.close()